In [ ]:
# wind exclusions
social_exclusion = snakemake.input.social_excl
environmental_exclusion = snakemake.input.env_excl
technical_exclusion = snakemake.input.techn_excl
onshore_slope = snakemake.input.onshore_slope

if snakemake.params.elevation_excl:
    heightshp = snakemake.input.elevation
else:
    heightshp = ""

if snakemake.params.onshore_slope_excl:
    onshore_slope = snakemake.input.onshore_slope
else:
    onshore_slope=""

# solar exclusions
solar_slope = snakemake.input.solar_slope
solar_env_excl = snakemake.input.solar_env_excl
solar_agr_excl = snakemake.input.solar_agr_excl
CORINE = snakemake.input.corine

CORINE = snakemake.input.corine

weatherdata = snakemake.input.weatherdata
desired_regions = snakemake.params.aggregated_regions

# Geodata files to use for selecting country onshore and offshore area:
geodata_files = {
    "onshore": snakemake.input.euroshape,
    "offshore_bottom": snakemake.input.eurooffshoreshape,
}

cfdata = snakemake.input.cfdata

grid_areas = snakemake.output.grid_areas

In [ ]:
panel = "CSi"
orientation = "latitude_optimal"

file_name = geodata_files["onshore"]

In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)

import io
import os
import pathlib

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import xarray as xr
from atlite.gis import ExclusionContainer, shape_availability
from shapely.geometry import Polygon

In [ ]:
atlite.__version__  # should be 0.2.4

In [ ]:
xr.__version__  # should be 0.18.2

In [ ]:
europe = gpd.read_file(geodata_files["onshore"]).set_index(["index"])
bad_zones = ['ES64','EL43']
europe = europe.query("index != @bad_zones")

In [ ]:
boundaries = []
for geodata_file_name, geodata_file_path in geodata_files.items():
    print(geodata_file_path)
    boundaries.append(gpd.read_file(geodata_file_path).set_index(["index"]))

boundaries = pd.concat(boundaries).bounds

In [ ]:
boundaries = boundaries.groupby(lambda x: "bountry").agg(
    {"minx": "min", "miny": "min", "maxx": "max", "maxy": "max"}
)

boundaries

In [ ]:
cutout = atlite.Cutout(path=weatherdata)
cutout.prepared_features
cutout.prepare()

In [ ]:
cutout = atlite.Cutout(
    path="../3_intermediate_data/intermediatecutout.nc",
    data=cutout.data.sel(
        x=slice(
            boundaries.loc["bountry", "minx"],
            boundaries.loc["bountry", "maxx"],
        ),
        y=slice(
            boundaries.loc["bountry", "miny"],
            boundaries.loc["bountry", "maxy"],
        ),
    ),
)

cutout.prepare()

In [ ]:
gridcellnamingfunction = (
    lambda x: "x"
    + (x.x * 100).astype("int").astype("str")
    + "y"
    + (x.y * 100).astype("int").astype("str")
)

if snakemake.wildcards.spatial == "grid":
    (
        cutout.grid.assign(gridcell=gridcellnamingfunction)
        .loc[:, ["gridcell"]]
        .to_csv(snakemake.output["indreg"], header=False, index=False)
    )
else:
    with open(snakemake.output["indreg"], "w"):
        pass

In [ ]:
def cutoff_raster(cf, cutoff, output):
    # if technology == 'pv':
    #     cf = cutout.pv(panel=panel, orientation=orientation, capacity_factor=True)
    #     smallestincluded = snakemake.params.cutoffs["solar"]

    # if technology == 'onwind':
    #     cf = cutout.wind(turbine=onshore_turbine, capacity_factor=True)
    #     smallestincluded = snakemake.params.cutoffs["onwind"]

    # if technology == 'offwind':
    #     cf = cutout.wind(turbine=offshore_bottom_turbine, capacity_factor=True)
    #     smallestincluded = snakemake.params.cutoffs["offwind"]

    excluded = xr.where(cf >= cutoff, 0, 1)

    # excluded = cf.where(cf.values>=cutoff, other=1)
    # excluded = excluded.where(cf.values<cutoff, other=0)
    cf_exclusion = excluded.rio.write_crs(europe.crs)

    if os.path.isfile(output):
        os.remove(output)

    cf_exclusion.rio.to_raster(output)

# Solar areas

In [ ]:
excluder_solar = ExclusionContainer()

In [ ]:
excluder_solar.add_raster(CORINE, codes=tuple(snakemake.params.corine_codes["solar"]))
excluder_solar.add_raster(solar_env_excl)
excluder_solar.add_raster(solar_agr_excl)

In [ ]:
if solar_slope != "":
    excluder_solar.add_raster(solar_slope)

In [ ]:
if heightshp != "":
    height = gpd.read_file(heightshp).to_crs(excluder_solar.crs)

    height = height[height["gridcode"] == 1]

    excluder_solar.add_geometry(height.geometry, invert=True)

In [ ]:
if snakemake.wildcards.spatial != "grid" and snakemake.params.cutoffs["solar"] != 0:
    cf = xr.open_dataarray(cfdata)

    low_cf = snakemake.output.cf_exclusion_solar
    cutoff_raster(
        cf.loc["Solar", :, :, :].mean(dim="time"),
        snakemake.params.cutoffs["solar"],
        low_cf,
    )
    excluder_solar.add_raster(low_cf)

In [ ]:
excluder_solar

In [ ]:
availability_matrix_solar = cutout.availabilitymatrix(
    europe, excluder_solar, nprocesses=snakemake.threads
)

In [ ]:
# plot eligible area - this is slow so commented out for now

# fig, ax = plt.subplots(figsize=(10,10))

# euro=europe.to_crs(excluder_solar.crs).dissolve().reset_index()

# excluder_solar.plot_shape_availability(euro,ax=ax)

In [ ]:
availability_matrix_solar

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_solar = availability_matrix_solar.stack(spatial=["x", "y"]) * area

# comparing the cap matrix before and after, the below is not required
# as all it does is reorder the array and breaks the latest xarray
# (v2024.03.0).

# capacity_matrix_solar = capacity_matrix_solar.reindex(
#    spatial=area.indexes.get("spatial")
# )


highRESareasSolar = (
    capacity_matrix_solar.unstack()
    .stack(spatial=["index", "x", "y"])
    .to_pandas()
    .reset_index()
    .replace()
    .assign(gridcell=gridcellnamingfunction)
    .rename(columns={0: "area"})
    .query("area != 0")
    .merge(europe, left_on="index", right_on="index")
    .rename(columns={"CNTR_CODE": "zone"})
    .loc[:, ["zone", "index", "x", "y", "area", "gridcell"]]
)

In [ ]:
if snakemake.params.include_rooftop:

    # Below code is necessary because ground mounted exclusions might have removed
    # certain NUTS2 entirely. Therefore we add back in 0 areas for the (maybe) missing
    # country/NUTS2/gridcell combos

    excluder_solar_rooftop = ExclusionContainer()
    excluder_solar_rooftop.add_raster(CORINE, codes=tuple([1,2,3]),invert=True)

    availability_matrix_solar_rooftop = cutout.availabilitymatrix(
        europe, excluder_solar_rooftop, nprocesses=snakemake.threads
    )
    
    area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6
    
    area = xr.DataArray(area, dims=("spatial"))
    
    capacity_matrix_solar_rooftop = availability_matrix_solar_rooftop.stack(spatial=["x", "y"]) * area

    highRESareasSolar_rooftop = (
        capacity_matrix_solar_rooftop.unstack()
        .stack(spatial=["index", "x", "y"])
        .to_pandas()
        .reset_index()
        .replace()
        .assign(gridcell=gridcellnamingfunction)
        .rename(columns={0: "area"})
        .query("area != 0")
        .merge(europe, left_on="index", right_on="index")
        .rename(columns={"CNTR_CODE": "zone"})
        .loc[:, ["zone", "index", "x", "y", "area", "gridcell"]]
    )
    
    highRESareasSolar_rooftop.area=highRESareasSolar_rooftop.area*0.
    
    highRESareasSolar=pd.concat((highRESareasSolar,highRESareasSolar_rooftop))
    
    highRESareasSolar=highRESareasSolar.groupby(["zone","index","x","y","gridcell"]).sum().reset_index()

In [ ]:
# if snakemake.wildcards.spatial == "region":
#     highRESareasSolar = (
#         capacity_matrix_solar.sum("spatial")
#         .to_pandas()
#         .reset_index()
#         .assign(
#             new_idx=lambda x: "Solar." + x["index"] + "." + x["index"],
#         )
#         .set_index("new_idx")
#         .drop(columns=["index"])
#         .loc[:, [0]]
#         .rename(columns={0: "area"})
#         .query("area != 0")
#     )

# Onshore wind areas

In [ ]:
excluder_wind_onshore = ExclusionContainer()

excluder_wind_onshore.add_raster(social_exclusion)
excluder_wind_onshore.add_raster(environmental_exclusion)
excluder_wind_onshore.add_raster(technical_exclusion)

if onshore_slope != "":
    excluder_wind_onshore.add_raster(onshore_slope)

In [ ]:
if snakemake.wildcards.spatial != "grid" and snakemake.params.cutoffs["onwind"] != 0:
    cf = xr.open_dataarray(cfdata)

    low_cf = snakemake.output.cf_exclusion_windon
    cutoff_raster(
        cf.loc["Windonshore", :, :, :].mean(dim="time"),
        snakemake.params.cutoffs["onwind"],
        low_cf,
    )
    excluder_wind_onshore.add_raster(low_cf)

In [ ]:
availability_matrix_wind_onshore = cutout.availabilitymatrix(
    europe, excluder_wind_onshore, nprocesses=snakemake.threads
)
availability_matrix_wind_onshore

In [ ]:
# plot eligible area - this is slow so commented out for now

#fig, ax = plt.subplots(figsize=(10,10))

#euro=europe.to_crs(excluder_wind_onshore.crs).dissolve().reset_index()

#excluder_wind_onshore.plot_shape_availability(euro,ax=ax)

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_onshore = (
    availability_matrix_wind_onshore.stack(spatial=["x", "y"]) * area
)

# comparing the cap matrix before and after, the below is not required
# as all it does is reorder the array and breaks the latest xarray
# (v2024.03.0).

# capacity_matrix_wind_onshore = capacity_matrix_wind_onshore.reindex(
#    spatial=area.indexes.get("spatial")
# )

highRESareasWindOnshore = (
    capacity_matrix_wind_onshore.unstack()
    .stack(spatial=["index", "x", "y"])
    .to_pandas()
    .reset_index()
    .replace()
    .assign(gridcell=gridcellnamingfunction)
    .rename(columns={0: "area"})
    .query("area != 0")
    .merge(europe, left_on="index", right_on="index")
    .rename(columns={"CNTR_CODE": "zone"})
    .loc[:, ["zone", "index", "x", "y", "area", "gridcell"]]
)

In [ ]:
# if snakemake.wildcards.spatial == "region":
#     highRESareasWindOnshore = (
#         capacity_matrix_wind_onshore.sum("spatial")
#         .to_pandas()
#         .reset_index()
#         .assign(
#             new_idx=lambda x: "Windonshore." + x["index"] + "." + x["index"],
#         )
#         .set_index("new_idx")
#         .drop(columns=["index"])
#         .loc[:, [0]]
#         .rename(columns={0: "area"})
#         .query("area != 0")
#     )

# Offshore wind areas

In [ ]:
europe_offshore_bottom = gpd.read_file(geodata_files["offshore_bottom"])
europe_offshore_bottom

In [ ]:
excluder_wind_offshore_bottom = ExclusionContainer()

In [ ]:
excluder_wind_offshore_bottom.add_raster(social_exclusion)
excluder_wind_offshore_bottom.add_raster(environmental_exclusion)
excluder_wind_offshore_bottom.add_raster(technical_exclusion)

In [ ]:
if snakemake.wildcards.spatial != "grid" and snakemake.params.cutoffs["offwind"] != 0:
    cf = xr.open_dataarray(cfdata)

    low_cf = snakemake.output.cf_exclusion_windoff
    cutoff_raster(
        cf.loc["Windoffshore", :, :, :].mean(dim="time"),
        snakemake.params.cutoffs["offwind"],
        low_cf,
    )
    excluder_wind_offshore_bottom.add_raster(low_cf)

In [ ]:
grid = cutout.grid.set_index(["x", "y"]).to_crs(3035)

europe_windoff = gpd.overlay(
    europe_offshore_bottom.reset_index().to_crs(3035), grid, how="intersection"
)
europe_windoff["centroid"] = europe_windoff["geometry"].centroid


zone = []
for i, p in europe_windoff.iterrows():
    ztemp = europe.loc[europe["CNTR_CODE"] == p["index"], :].to_crs(3035)
    dists = ztemp.distance(p["centroid"]).sort_values()

    if len(dists) == 0:
        break

    zone.append(dists.index[0])

europe_windoff["zone"] = zone

# europe_windoff.loc[:,["index","zone","geometry"]].to_file("europe_windoff.geojson")

In [ ]:
europe_offshore_bottom = (
    europe_windoff.loc[:, ["index", "zone", "geometry"]]
    .dissolve(by="zone")
    .rename(columns={"index": "zone"})
    .rename_axis(index={"zone": "index"})
)

In [ ]:
availability_matrix_wind_offshore_bottom = cutout.availabilitymatrix(
    europe_offshore_bottom, excluder_wind_offshore_bottom, nprocesses=snakemake.threads
)
availability_matrix_wind_offshore_bottom

In [ ]:
area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_offshore_bottom = (
    availability_matrix_wind_offshore_bottom.stack(spatial=["x", "y"]) * area
)

# comparing the cap matrix before and after, the below is not required
# as all it does is reorder the array and breaks the latest xarray
# (v2024.03.0).

# capacity_matrix_wind_offshore_bottom = capacity_matrix_wind_offshore_bottom.reindex(
#    spatial=area.indexes.get("spatial")
# )

highRESareasWindOffshoreBottom = (
    capacity_matrix_wind_offshore_bottom.unstack()
    .stack(spatial=["index", "x", "y"])
    .to_pandas()
    .reset_index()
    .replace()
    .assign(gridcell=gridcellnamingfunction)
    .rename(columns={0: "area"})
    .query("area != 0")
    .merge(europe, left_on="index", right_on="index")
    .rename(columns={"CNTR_CODE": "zone"})
    .loc[:, ["zone", "index", "x", "y", "area", "gridcell"]]
)

In [ ]:
highRESareas = pd.concat(
    (
        highRESareasSolar.round(1).assign(Tech="Solar"),
        highRESareasWindOnshore.round(1).assign(Tech="Windonshore"),
        highRESareasWindOffshoreBottom.round(1).assign(Tech="Windoffshore"),
    )
)

highRESareas.to_csv(grid_areas, index=False)

In [ ]:
# if snakemake.wildcards.spatial == "region":
#     highRESareasWindOffshoreBottom = (
#         capacity_matrix_wind_offshore_bottom.sum("spatial")
#         .to_pandas()
#         .reset_index()
#         .assign(
#             new_idx=lambda x: "Windoffshore." + x["index"] + "." + x["index"],
#         )
#         .set_index("new_idx")
#         .drop(columns=["index"])
#         .loc[:, [0]]
#         .rename(columns={0: "area"})
#         .query("area != 0")
#     )